In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
from tqdm.auto import tqdm
from matplotlib.colors import LogNorm
import pickle


cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
base_dir = os.path.dirname(os.path.dirname(cwd))
src_dir = base_dir + "/src"

sys.path.insert(0, src_dir)

from calibration import MPEM, MPEM_AVAILABLE, LinearNet, ActuationNet, DirectNet, PotentialNet, DirectGBT
from evaluate import ModelPhantom, EvaluationPackage, get_affine_fits

## Define paths

In [ ]:
#############################
######## Which eMNS? ########
#############################

emns = "octomag" # "octomag" or "navion"

if emns == "octomag":
    poly_fits_dir = parent_dir + "/gradients/data/"
    data_dir = base_dir + "/data/octomag_data/split_dataset/"
    nn_params_dir = parent_dir + "/training/params/"
    gbt_params_dir = parent_dir + "/training/trees/"
    assembled_data_dir = cwd + "/evaluation_packages/"
    mpem_params_dir = base_dir + "/mpem/optimized_parameters/"
elif emns == "navion":
    poly_fits_dir = parent_dir + "/gradients/data_navion/"
    data_dir = base_dir + "/data/navion_data/split_dataset/"
    nn_params_dir = parent_dir + "/training/params_navion/"
    gbt_params_dir = parent_dir + "/training/trees/navion_trees/"
    assembled_data_dir = cwd + "/evaluation_packages/navion/"
    mpem_params_dir = base_dir + "/mpem/mpem_navion/optimized_parameters/"

## Load evaluation data package

In [ ]:
##############################
######## Package name ########
##############################
test_package_name = "test_eval_pack.pkl"
training_package_name = "train_eval_pack.pkl"

# Check if test set package exists
test_package_path = assembled_data_dir + test_package_name
if os.path.exists(test_package_path):
    print(f"Loading existing test set evaluation package from {test_package_path}...") 
    test_package = EvaluationPackage.load_from(test_package_path)
else:
    # Create new
    print(f"Creating new test set evaluation package...")
    test_package = EvaluationPackage()

# Check if training set package exists
training_package_path = assembled_data_dir + training_package_name
if os.path.exists(training_package_path):
    print(f"Loading existing training set evaluation package from {training_package_path}...") 
    training_package = EvaluationPackage.load_from(training_package_path)
else:
    # Create new
    print(f"Creating new training set evaluation package...")
    training_package = EvaluationPackage()

## Load new models to gather evals from

In [ ]:
# Load models to session
nn_models = [
    ActuationNet.load_from(nn_params_dir + "ActuationNet_100_256x256.pt"),
    ActuationNet.load_from(nn_params_dir + "ActuationNet_100_256x256x256.pt"),
    ActuationNet.load_from(nn_params_dir + "ActuationNet_100_512x512x512.pt"),
    ActuationNet.load_from(nn_params_dir + "ActuationNet_50_512x512x512.pt"),
    ActuationNet.load_from(nn_params_dir + "ActuationNet_20_512x512x512.pt"),
    ActuationNet.load_from(nn_params_dir + "ActuationNet_5_512x512x512.pt"),
    ActuationNet.load_from(nn_params_dir + "ActuationNet_1_512x512x512.pt"),
    DirectNet.load_from(nn_params_dir + "/DirectNet_100_256x256.pt"),
    DirectNet.load_from(nn_params_dir + "/DirectNet_100_256x256x256.pt"),
    DirectNet.load_from(nn_params_dir + "/DirectNet_100_512x512x512.pt"),
    DirectNet.load_from(nn_params_dir + "/DirectNet_50_512x512x512.pt"),
    DirectNet.load_from(nn_params_dir + "/DirectNet_20_512x512x512.pt"),
    DirectNet.load_from(nn_params_dir + "/DirectNet_5_512x512x512.pt"),
    DirectNet.load_from(nn_params_dir + "/DirectNet_1_512x512x512.pt"),
    PotentialNet.load_from(nn_params_dir + "/PotentialNet_100_256x256.pt"),
    PotentialNet.load_from(nn_params_dir + "/PotentialNet_100_256x256x256.pt"),
    PotentialNet.load_from(nn_params_dir + "/PotentialNet_100_512x512x512.pt"),
    PotentialNet.load_from(nn_params_dir + "/PotentialNet_50_512x512x512.pt"),
    PotentialNet.load_from(nn_params_dir + "/PotentialNet_20_512x512x512.pt"),
    PotentialNet.load_from(nn_params_dir + "/PotentialNet_5_512x512x512.pt"),
    PotentialNet.load_from(nn_params_dir + "/PotentialNet_1_512x512x512.pt"),
]

tree_models = [
    DirectGBT.load_from(gbt_params_dir + "/DirectGBT_100_32.gbt.zip"),
    DirectGBT.load_from(gbt_params_dir + "/DirectGBT_100_64.gbt.zip"),
    DirectGBT.load_from(gbt_params_dir + "/DirectGBT_100_128.gbt.zip"),
    DirectGBT.load_from(gbt_params_dir + "/DirectGBT_50_128.gbt.zip"),
    DirectGBT.load_from(gbt_params_dir + "/DirectGBT_20_128.gbt.zip"),
    DirectGBT.load_from(gbt_params_dir + "/DirectGBT_5_128.gbt.zip"),
    DirectGBT.load_from(gbt_params_dir + "/DirectGBT_1_128.gbt.zip"),
]


mpem_models = []
if MPEM_AVAILABLE :
    
    mpem_param_files = {
        "MPEM_5_3": "optimized_octopole_5.yaml",
        "MPEM_1_3": "optimized_octopole_1.yaml",
        "MPEM_5_2": "optimized_quadrupole_5.yaml",
        "MPEM_5_1": "optimized_dipole_5.yaml",
    }

    for model_name, param_file in mpem_param_files.items(): 
        mpem_models.append(MPEM(mpem_params_dir + param_file, model_name))


models = nn_models + tree_models + mpem_models

In [ ]:
# Load models into package
for model in models:
    test_package.load_model(model, force=True)
    training_package.load_model(model, force=True)

## Load data into package

In [ ]:
training_data_base = "training_data"
test_data_base = "test_data"

data_percentages = [100, 50, 20, 5, 1]

overwrite = True


# Load test data. No reduction here.
test_data = pd.read_pickle(data_dir + test_data_base + ".pkl"   )
pos_cols = ['x', 'y', 'z']
current_cols = [col for col in test_data.columns if col.startswith('em_')]
field_cols = ['Bx', 'By', 'Bz']

# Extract positions, currents, and fields cols from dataframes
test_positions = test_data[pos_cols].to_numpy()
test_currents = test_data[current_cols].to_numpy()
test_fields = test_data[field_cols].to_numpy()

# Load data into package
print(f"Loading full test data into evaluation package...")
test_package.load_features_and_labels(test_positions, test_currents, test_fields, dataset_percentage=100, overwrite=overwrite)


for percentage in data_percentages:
    training_data_file = f"{training_data_base}_{percentage}.pkl" if percentage != 100 else f"{training_data_base}.pkl"

    training_data = pd.read_pickle(data_dir + training_data_file)

    # Extract positions, currents, and fields cols from dataframes  
    training_positions = training_data[pos_cols].to_numpy()
    training_currents = training_data[current_cols].to_numpy()
    training_fields = training_data[field_cols].to_numpy()

    # Load data into package
    print(f"Loading training data with percentage {percentage} into training package...")
    training_package.load_features_and_labels(training_positions, training_currents, training_fields, dataset_percentage=percentage, overwrite=overwrite)

## Compute field predictions

In [ ]:
# For test dataset
test_package.compute_field_predictions(keep_existing=True)

# Store package
print("Storing test evaluation package...\n")
test_package.store(assembled_data_dir + test_package_name)
print("Successfully stored test evaluation package.")

In [ ]:
# For training dataset
training_package.compute_field_predictions(keep_existing=True)

# Store package
print("Storing training evaluation package...\n")   
training_package.store(assembled_data_dir + training_package_name)
print("Successfully stored training evaluation package.")

## Compute gradient predictions

In [ ]:
# For test dataset
test_package.compute_gradient_predictions(keep_existing=False)

# Store package
print("Storing test evaluation package with gradient predictions...\n")
test_package.store(assembled_data_dir + test_package_name)
print("Successfully stored test evaluation package with gradient predictions.")

In [ ]:
# For training dataset
training_package.compute_gradient_predictions(keep_existing=False)

# Store package
print("Storing training evaluation package...\n")
training_package.store(assembled_data_dir + training_package_name)
print("Successfully stored training evaluation package.")

## Load gradient references from polynomial fits

Only required if more complex gradient evaluation metrics than curl magnitude and divergence are used.

In the paper, and in this project's evaluation notebooks, we use only these metrics, and therefore do not require gradients references from polynomial fits to be computed. In any case, these can be obtained using the notebook in `notebooks/gradients/poly_fit.ipynb`.

In [ ]:
test_set_poly_fits_file = "poly_grads_test.pkl"
training_set_poly_fits_file = "poly_grads_training.pkl"

# Load polynomial fits
try:
    with open(poly_fits_dir + test_set_poly_fits_file, "rb") as f:
        test_set_poly_fits = pickle.load(f)
    with open(poly_fits_dir + training_set_poly_fits_file, "rb") as f:  
        training_set_poly_fits = pickle.load(f)

    # Choose which order polynomial fit, and nearest neighbor support count to use
    order = 1
    nn_count = 125

    # Add polynomial fits to packages
    test_package.load_gradient_references(test_set_poly_fits)
    training_package.load_gradient_references(training_set_poly_fits)

    # Store packages with polynomial fit references
    print("Storing test evaluation package with polynomial fit references...\n")
    test_package.store(assembled_data_dir + test_package_name)
    print("Successfully stored test evaluation package with polynomial fit references.")
    print("Storing training evaluation package with polynomial fit references...\n")
    training_package.store(assembled_data_dir + training_package_name)
    print("Successfully stored training evaluation package with polynomial fit references.")
except Exception as e:
    print(f"Error loading polynomial fits: {e}. Proceding without adding polynomial fit references.")

## Get and load affine fits per position for reference

Useful for computing floor metrics for a "perfect" affine map estimator. Was not used in the paper.

In [ ]:
# Compute affine fit for each position in test set?
recompute_test_affine_fits = False

if recompute_test_affine_fits:
    test_preds, test_fits = get_affine_fits(test_data, resolution=float(0.005/3.0))

    test_affine_phantom = ModelPhantom(name="AffinePerPos")

    test_package.manual_load_field_predictions(test_affine_phantom, test_preds)

    # Store package
    print("Storing test evaluation package with affine per position fits...\n")
    test_package.store(assembled_data_dir + test_package_name)
    print("Successfully stored test evaluation package with affine per position fits.")

In [ ]:
# Compute affine fit for each position in training set?
recompute_training_affine_fits = False

if recompute_training_affine_fits:
    training_preds, training_fits = get_affine_fits(training_data, resolution=float(0.005/3.0))

    training_affine_phantom = ModelPhantom(name="AffinePerPos")

    training_package.manual_load_field_predictions(training_affine_phantom, training_preds)
    # Store package
    print("Storing training evaluation package with affine per position fits...\n")
    training_package.store(assembled_data_dir + training_package_name)
    print("Successfully stored training evaluation package with affine per position fits.")